In [2]:
from apyori import apriori
import pandas as pd

C:\Users\User\AppData\Local\Temp\ipykernel_17860\350574535.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [15]:
df = pd.read_csv('market.csv', sep=';')
df.drop(['Date', 'Price', 'CustomerID', 'Country'], axis=1, inplace=True)
df.sample(10)

C:\Users\User\AppData\Local\Temp\ipykernel_17860\2818753376.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('market.csv', sep=';')


,BillNo,Itemname,Quantity
376624,570676,SILVER BELLS TABLE DECORATION,12
385032,571328,BALLOON PUMP WITH 10 BALLOONS,8
281695,562553,CHERRY BLOSSOM PASSPORT COVER,1
419678,574165,WOOD STAMP SET THANK YOU,12
460618,577095,BAG 500g SWIRLY MARBLES,1
131219,547924,IVORY HANGING DECORATION HEART,24
189395,553830,SCANDINAVIAN PAISLEY PICNIC BAG,20
194801,554326,HOME SMALL WOOD LETTERS,1
135951,548492,SET 3 PAPER VINTAGE CHICK PAPER EGG,1
414744,573600,RABBIT NIGHT LIGHT,48


In [16]:
df['BillNo'].unique().shape

(21665,)

In [26]:
df['Itemname'].isna().sum()

1455

In [27]:
df['Itemname'] = df['Itemname'].fillna(df['Itemname'].mode()[0])

In [28]:
transactions_dict = {bill: [] for bill in df['BillNo']}
for row in df.iterrows():
    transactions_dict[row[1]['BillNo']].append(row[1]['Itemname'])
transactions_dict

{536365: ['WHITE HANGING HEART T-LIGHT HOLDER',
  'WHITE METAL LANTERN',
  'CREAM CUPID HEARTS COAT HANGER',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.',
  'SET 7 BABUSHKA NESTING BOXES',
  'GLASS STAR FROSTED T-LIGHT HOLDER'],
 536366: ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT'],
 536367: ['ASSORTED COLOUR BIRD ORNAMENT',
  "POPPY'S PLAYHOUSE BEDROOM",
  "POPPY'S PLAYHOUSE KITCHEN",
  'FELTCRAFT PRINCESS CHARLOTTE DOLL',
  'IVORY KNITTED MUG COSY',
  'BOX OF 6 ASSORTED COLOUR TEASPOONS',
  'BOX OF VINTAGE JIGSAW BLOCKS',
  'BOX OF VINTAGE ALPHABET BLOCKS',
  'HOME BUILDING BLOCK WORD',
  'LOVE BUILDING BLOCK WORD',
  'RECIPE BOX WITH METAL HEART',
  'DOORMAT NEW ENGLAND'],
 536368: ['JAM MAKING SET WITH JARS',
  'RED COAT RACK PARIS FASHION',
  'YELLOW COAT RACK PARIS FASHION',
  'BLUE COAT RACK PARIS FASHION'],
 536369: ['BATH BUILDING BLOCK WORD'],
 536370: ['ALARM CLOCK BAKELIKE PINK',
  'ALARM CLOCK BAKELIKE RED',
  'ALARM CLOCK BAKELIKE

In [32]:
min_conf = 0.1
min_support = 0.002
min_lift = 2
min_length = 2
max_length = 2
rules = apriori(transactions=list(transactions_dict.values()),
                min_confidence=min_conf,
                min_lift=min_lift,
                min_length=min_length,
                min_support=min_support,
                max_length=max_length)
result = list(rules)
result[:10]

[RelationRecord(items=frozenset({'10 COLOUR SPACEBOY PEN', '12 PENCIL SMALL TUBE WOODLAND'}), support=0.002215555042695592, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10 COLOUR SPACEBOY PEN'}), items_add=frozenset({'12 PENCIL SMALL TUBE WOODLAND'}), confidence=0.15384615384615385, lift=9.44214425800828), OrderedStatistic(items_base=frozenset({'12 PENCIL SMALL TUBE WOODLAND'}), items_add=frozenset({'10 COLOUR SPACEBOY PEN'}), confidence=0.13597733711048157, lift=9.44214425800828)]),
 RelationRecord(items=frozenset({'10 COLOUR SPACEBOY PEN', '4 TRADITIONAL SPINNING TOPS'}), support=0.0023078698361412415, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10 COLOUR SPACEBOY PEN'}), items_add=frozenset({'4 TRADITIONAL SPINNING TOPS'}), confidence=0.16025641025641024, lift=7.482661914235189), OrderedStatistic(items_base=frozenset({'4 TRADITIONAL SPINNING TOPS'}), items_add=frozenset({'10 COLOUR SPACEBOY PEN'}), confidence=0.10775862068965517, lift=7.48266191423519)

In [61]:
result[0].ordered_statistics[0].lift

9.44214425800828

In [64]:
result_df = pd.DataFrame(
    data={'Support': [rule.support for rule in result],
          'Confidence': [rule.ordered_statistics[0].confidence for rule in result],
          'Lift': [rule.ordered_statistics[0].lift for rule in result]
          }
)
result_df.sample(10)

,Support,Confidence,Lift
6118,0.002954,0.148837,3.334600
17897,0.002631,0.135714,3.984079
1219,0.002908,0.198113,5.103594
38485,0.003508,0.207650,4.131077
54819,0.002169,0.112440,4.013207
23570,0.003785,0.158915,2.557866
24337,0.002308,0.174216,7.563908
22221,0.002862,0.160207,5.172695
57255,0.003323,0.199446,3.982486
27454,0.002400,0.337662,6.194288


In [66]:
import plotly.express as px
fig = px.imshow(
    result_df.corr(), aspect='auto', text_auto=True
)
fig.show()